In [4]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as BeautifulSoup

verbose = True # abilita descrizione stato dello scraping

n = 1 # contatore
url_cat = ['donna', 'uomo'] # categorie url nerogiardini
url_pages = ['', '/saldi']
# liste dati
prod_id = []
prod_category = []
prod_name = []
prod_price = []
prod_sales = []
prod_colors = []
prod_sex = []

# itero per categorie
for page in url_pages:
    for cat in url_cat:
        for i in range(1, 100):  # scorrimento tra 100 pagine per il paramtro p=
            url = f'https://nerogiardini.it/it/{cat}{page}/calzature?p=' + str(i)
            req = requests.get(url)
            
            if req.status_code != 200:
                if verbose:
                    print(f'Page: {url} --> \033[41mStatus Code {req.status_code}\033[0m')
                break
            
            soup = BeautifulSoup(req.text)
            all_prod = soup.select('li.item.product')
            
            if len(all_prod) == 0: # break in caso non ci sono più prodotti
                break;
            
            for prod in all_prod:
                prod_id.append(prod['class'][-1])
                prod_name.append(prod.findAll('h2')[0].get_text(strip=True))
                prod_category.append(prod_name[-1].lower().split(' ' + cat)[0].capitalize())

                price = float(prod.findAll('span', class_='price')[0].get_text(strip=True).replace('\xa0€','').replace(',', '.'))
                
                if 'saldi' in page:
                    prod_sales.append(price)
                    prod_price.append(None)
                else:    
                    prod_price.append(price)
                    prod_sales.append(None)
                
                prod_sex.append(cat.capitalize())
                colors_variant = prod.select('div.variant')        
                color_list = []
                if len(colors_variant):
                    for color in colors_variant:
                        color_list.append(color['alt'])
                else:
                    color_list = None
                prod_colors.append(color_list)
    
                if verbose:
                    print(n, prod_id[-1], prod_category[-1], prod_name[-1], prod_price[-1], prod_sales[-1], prod_colors[-1],  sep='\t')
                    n+=1
            if verbose:
                print(f'Page {i}: {url} --> \033[42mStatus OK\033[0m')

1	25P_E011041DE	Décolleté	Décolleté Donna in Vernice	149.5	None	None
2	25P_E011073DE	Décolleté	Décolleté Donna in Pelle	149.5	None	['Nero', 'Rosa']
3	25P_E011084DE	Décolleté	Décolleté Donna in Pelle	149.5	None	None
4	25P_E011085DE	Décolleté	Décolleté Donna in Vernice	149.5	None	None
5	25P_E012830DE	Sandali	Sandali Donna in Materiale Tecnico	179.5	None	['Marrone', 'Rosa', 'Argento']
6	25P_E115530D	Sandali	Sandali Donna in Suede	119.5	None	['Nero', 'Beige']
7	25P_E211041DE	Décolleté	Décolleté Donna in Vernice	149.5	None	['Nero', 'Rosso']
8	25P_E211070DE	Décolleté	Décolleté Donna in Pelle	145.0	None	None
9	25P_E211072DE	Décolleté	Décolleté Donna in Pelle	149.5	None	['Nero', 'Rosa']
10	25P_E218210D	Sabot	Sabot Donna in Pelle	129.5	None	None
11	25P_E218212D	Mocassini	Mocassini Donna in Pelle	135.0	None	None
12	25P_E218331DE	Décolleté	Décolleté Donna in Pelle	149.5	None	None
Page 1: https://nerogiardini.it/it/donna/calzature?p=1 --> Status OK
13	25P_E218332DE	Décolleté	Décolleté Donna in Pel

In [6]:
# creazione del dataframe
nero_dict = {'id': prod_id, 'category': prod_category, 'name': prod_name, 'price': prod_price, 'sales_price' : prod_sales, 'color':prod_colors, 'sex': prod_sex} 
nero_df = pd.DataFrame(nero_dict)

# sostituzione della categoria con typing errato 'Snekares'
nero_df.category = nero_df.category.replace('Snekares', 'Sneakers')
nero_df.category = nero_df.category.replace('Sneakers alta', 'Sneakers') 

nero_df = nero_df.explode('color').reset_index(drop=True).drop_duplicates(subset=['id', 'color'])
nero_df.to_csv('nerogiard_scrap_250225.csv')
nero_df

,id,category,name,price,sales_price,color,sex
0,25P_E011041DE,Décolleté,Décolleté Donna in Vernice,149.5,NaN,None,Donna
1,25P_E011073DE,Décolleté,Décolleté Donna in Pelle,149.5,NaN,Nero,Donna
2,25P_E011073DE,Décolleté,Décolleté Donna in Pelle,149.5,NaN,Rosa,Donna
3,25P_E011084DE,Décolleté,Décolleté Donna in Pelle,149.5,NaN,None,Donna
4,25P_E011085DE,Décolleté,Décolleté Donna in Vernice,149.5,NaN,None,Donna
...,...,...,...,...,...,...,...
1166,24A_I102155U,Sneakers,Sneakers Uomo in Pelle,NaN,125.5,None,Uomo
1167,24A_I400458U,Sneakers,Sneakers Alta Uomo in Pelle,NaN,125.5,None,Uomo
1168,24A_I400515U,Stivaletti,Stivaletti Uomo in Pelle,NaN,132.5,None,Uomo
1169,24A_I202640U,Anfibi,Anfibi Uomo in Pelle,NaN,136.5,Nero,Uomo
